In [1]:
"""
Simple test for formula loading and evaluation.
"""
import os
import sys
sys.path.insert(0, os.path.abspath(os.path.join("/home/ubuntu/projects/hindsight/examples/data/ast", '../../..')))

import matplotlib.pyplot as plt
import jax
import pandas as pd
import pandas_ta as ta
import jax.numpy as jnp

from src import DataManager
from src.data.ast import parse_formula
from src.data.ast.manager import FormulaManager
from src.data.ast.functions import register_built_in_functions, get_function_context
from src.data.core import prepare_for_jit

/home/ubuntu/projects/hindsight/src/data/core/struct.py:289: AccessorRegistrationWarning: registration of accessor <class 'src.data.core.struct.DataArrayDateTimeAccessor'> under name 'dt' for type <class 'xarray.core.dataarray.DataArray'> is overriding a preexisting attribute with the same name.
  @xr.register_dataarray_accessor('dt')


In [2]:
# Register built-in functions
register_built_in_functions()

In [3]:
# Initialize and load formulas
manager = FormulaManager()

In [4]:
# Load CRSP data

dm = DataManager()
configs = dm.get_builtin_configs()

In [5]:
configs

['equity_standard']

In [6]:
ds = dm.load_builtin("equity_standard")

wrds/equity/crsp: Attemping to load found cache(/home/suchismit/data/cache/wrds/equity/crsp/4f5bc4c841f1d1d0506b704fdad82df5_2000-01-01_2023-12-01.nc).
wrds/equity/crsp: Successfully loaded from /home/suchismit/data/cache/wrds/equity/crsp/4f5bc4c841f1d1d0506b704fdad82df5_2000-01-01_2023-12-01.nc


In [7]:
ds = ds['equity_prices']

In [8]:
list(ds.data_vars.keys())

['cusip',
 'issuno',
 'hexcd',
 'hsiccd',
 'bidlo',
 'askhi',
 'prc',
 'vol',
 'ret',
 'bid',
 'ask',
 'shrout',
 'cfacpr',
 'cfacshr',
 'altprc',
 'spread',
 'altprcdt',
 'retx',
 'comnam',
 'exchcd',
 'me']

In [9]:
ds_jit, _ = prepare_for_jit(ds)

In [10]:
function_context = get_function_context() 

In [11]:
manager = FormulaManager()

In [12]:
manager.list_formulas()

['alma',
 'chcsho_12m',
 'chcsho_1m',
 'chcsho_3m',
 'chcsho_6m',
 'dema',
 'div12m_me',
 'div1m_me',
 'div3m_me',
 'div6m_me',
 'dividend',
 'dividend_times_shares',
 'fwma',
 'hma',
 'hwma',
 'kama',
 'market_equity',
 'prc_adj',
 'price_ret',
 'ret_12m',
 'ret_1m',
 'ret_3m',
 'ret_6m',
 'rsi',
 'shares_adj',
 'simple_rsi_momentum',
 'triple_exponential_smoothing',
 'ts_combined',
 'ts_momentum',
 'ts_signal',
 'wma']

In [13]:
ds_jit

<xarray.Dataset> Size: 468MB
Dimensions:       (year: 24, month: 12, day: 1, asset: 22555, time: 288)
Coordinates:
  * year          (year) int32 96B 2000 2001 2002 2003 ... 2020 2021 2022 2023
  * month         (month) int32 48B 1 2 3 4 5 6 7 8 9 10 11 12
  * day           (day) int64 8B 1
  * asset         (asset) float64 180kB 1e+04 1e+04 ... 9.344e+04 9.344e+04
    time          (year, month, day) datetime64[ns] 2kB 2000-01-01 ... 2023-1...
    mask          (time) bool 288B True True True True ... True True True True
    mask_indices  (time) int64 2kB 0 1 2 3 4 5 6 ... 281 282 283 284 285 286 287
    permco        (asset) float32 90kB 7.953e+03 7.954e+03 ... 5.345e+04
Data variables: (12/19)
    issuno        (year, month, day, asset) float32 26MB 1.04e+04 ... 6.625e+04
    hexcd         (year, month, day, asset) float32 26MB 2.0 3.0 3.0 ... nan 3.0
    hsiccd        (year, month, day, asset) float32 26MB 4.925e+03 ... 9.999e+03
    bidlo         (year, month, day, asset) float32 26MB 8.0 11.62 ... nan 235.6
    askhi         (year, month, day, asset) float32 26MB 8.562 13.0 ... 261.4
    prc           (year, month, day, asset) float32 26MB 8.125 11.88 ... 248.5
    ...            ...
    altprc        (year, month, day, asset) float32 26MB 8.125 11.88 ... 248.5
    spread        (year, month, day, asset) float32 26MB nan nan nan ... nan nan
    altprcdt      (year, month, day, asset) float32 26MB 1.464e+04 ... 2.337e+04
    retx          (year, month, day, asset) float32 26MB -0.04412 ... 0.03499
    exchcd        (asset) float64 180kB 2.0 3.0 3.0 3.0 3.0 ... 3.0 3.0 3.0 3.0
    me            (year, month, day, asset) float32 26MB 1.991e+04 ... 7.899e+08

In [14]:
@jax.jit
def _eval(): 
    context = {
        '_dataset': ds, # captured by closure
        'me': 'me',
        'ret': 'ret',
        'cfacpr': 'cfacpr',
        'prc': 'prc',
        'retx': 'retx',
        'shrout': 'shrout',
        'cfacshr': 'cfacshr'   
    }
    
    formulas_to_eval = [
        'market_equity',
        'chcsho_12m',
        'chcsho_1m',
        'chcsho_3m',
        'chcsho_6m',
        'div12m_me',
        'div1m_me',
        'div3m_me',
        'div6m_me',
        'dividend',
        'dividend_times_shares',
        'prc_adj',
        'price_ret',
        'ret_12m',
        'ret_1m',
        'ret_3m',
        'ret_6m',
        'shares_adj',]

    result = manager.evaluate_bulk(formulas_to_eval, context)
    return result

In [15]:
res = _eval()

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


In [16]:
res

<xarray.Dataset> Size: 468MB
Dimensions:                (year: 24, month: 12, day: 1, asset: 22555, time: 288)
Coordinates:
  * year                   (year) int32 96B 2000 2001 2002 ... 2021 2022 2023
  * asset                  (asset) float64 180kB 1e+04 1e+04 ... 9.344e+04
    mask_indices           (time) int64 2kB 0 1 2 3 4 5 ... 283 284 285 286 287
  * month                  (month) int32 48B 1 2 3 4 5 6 7 8 9 10 11 12
  * day                    (day) int64 8B 1
    permco                 (asset) float32 90kB 7.953e+03 ... 5.345e+04
    mask                   (time) bool 288B True True True ... True True True
    time                   (year, month, day) datetime64[ns] 2kB 2000-01-01 ....
Data variables: (12/18)
    chcsho_12m             (year, month, day, asset) float32 26MB ...
    chcsho_1m              (year, month, day, asset) float32 26MB ...
    chcsho_3m              (year, month, day, asset) float32 26MB ...
    chcsho_6m              (year, month, day, asset) float32 26MB ...
    div12m_me              (asset, year, month, day) float32 26MB ...
    div1m_me               (asset, year, month, day) float32 26MB ...
    ...                     ...
    price_ret              (year, month, day, asset) float32 26MB ...
    ret_12m                (year, month, day, asset) float32 26MB ...
    ret_1m                 (year, month, day, asset) float32 26MB ...
    ret_3m                 (year, month, day, asset) float32 26MB ...
    ret_6m                 (year, month, day, asset) float32 26MB ...
    shares_adj             (year, month, day, asset) float32 26MB ...

In [17]:
res['dividend'].sel(asset=14593).data

Array([[[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
        [0.        ],
    